## Search and manipulate .docx files

In [1]:
import os
import docx
from docx import Document
import re
import shutil
from shutil import copyfile

In [2]:
pwd

'C:\\Users\\user\\Desktop\\practice_in_2020'

In [3]:
# Change current working directory
os.chdir(r'C:\\Users\\user\\Desktop\\doc_files')

In [4]:
pwd

'C:\\Users\\user\\Desktop\\doc_files'

In [5]:
# A function that extracts files that match the pattern
def search_file(filename, pattern):
    doc = docx.Document(filename)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    if re.findall(pattern, str(text)) != []:
        return filename
    else:
        return 0

In [15]:
# Search docx files with content matching particular pattern
# Search for companies that are in Taipei
pattern = input('Please enter the keyword')
#pattern = '臺北'
matched_files = []
for file in os.listdir():
    if not file.endswith('.docx'): continue
    x = search_file(file, pattern)
    if x != 0:
        matched_files.append(x)

Please enter the keyword臺北


In [16]:
# Extract matched files path
matched_files_paths = []
for matched_file in matched_files:
    path = f"\\Users\\user\\Desktop\\doc_files\\{matched_file}"
    matched_files_paths.append(path)

In [17]:
# A function that manipulates files (move, copy, or delete)
def file_manipulate(mode, matched_file_path, to):
    if mode == 'move':
        shutil.move(matched_file_path, to)
    elif mode == 'copy':
        shutil.copy(matched_file_path, to)
    elif mode == 'delete':
        os.remove(matched_file_path)        
    else:
        return 'no such mode'

In [18]:
# Manipulate matched files
mode = input("move, copy, or delete?")
to = input("Please provide the file path")
#mode = 'copy'
#to = '\Users\user\Desktop\new_doc_files'

if mode not in ['move','copy', 'delete']:
    print('Warning: Invalid mode')
else:
    for matched_file_path in matched_files_paths:
        file_manipulate(mode, matched_file_path, to)    

move, copy, or delete?copy
Please provide the file path\Users\user\Desktop\new_doc_files


In [19]:
# Change current working directory
os.chdir(r'C:\\Users\\user\\Desktop\\new_doc_files')

In [20]:
# A function that gets text in files
def get_text(filename):
    doc = docx.Document(filename)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    return '_'.join(text) 

In [21]:
# Get all company names
company_names = []
for matched_file in matched_files:
    pat = re.compile(r"(?<=公司名稱)(\t+\w+)(?=章程)")
    result = pat.search(get_text(matched_file))
    company_name = result.group(0).replace('\t', '')
    company_names.append(company_name)

In [22]:
# Get all director names
director_names = []
for matched_file in matched_files:
    pat = re.compile(r"(?<=代表人姓名)(\t+\w+)(?=_)")
    result = pat.search(get_text(matched_file))
    director_name = result.group(0).replace('\t', '')
    director_names.append(director_name)

## Make invitation card

In [23]:
from docx.oxml.ns import qn
from docx.shared import Pt
icf = docx.Document('invitation_card.docx')

In [24]:
def get_para_data(output_doc_name, paragraph):
    output_para = output_doc_name.add_paragraph()
    for run in paragraph.runs:
        output_run = output_para.add_run(run.text)
        output_run.bold = run.bold
        output_run.italic = run.italic
        output_run.font.name = '標楷體'
        output_run._element.rPr.rFonts.set(qn('w:eastAsia'), u'標楷體')
        output_run.font.size = run.font.size
        output_run.underline = run.underline
    output_para.paragraph_format.alignment = paragraph.paragraph_format.alignment

In [25]:
def output(icf_docx,name,company):
    output_doc = Document()
    for para in icf.paragraphs:
        get_para_data(output_doc, para)
    a = ["１１１"]
    for i in a:
        for p in output_doc.paragraphs:
            if p.text.find(i)>=0:
                p.text=p.text.replace(i,name)
    b = ["２２２"]
    for i in b:
        for p in output_doc.paragraphs:
            if p.text.find(i)>=0:
                p.text=p.text.replace(i,company)
    for i,paragraph in enumerate(output_doc.paragraphs):
        if i==5 or i==6:
            for run in paragraph.runs:
                run.font.name = '標楷體'
                run._element.rPr.rFonts.set(qn('w:eastAsia'), u'標楷體')
                run.font.size = Pt(22)
    return output_doc.save(str(j)+'_OutputDoc.docx')

In [26]:
for j in range(0,len(company_names)):
    output(icf,director_names[j],company_names[j])